<h2>Convert all xml to csv</h2>
<p>Traverse through the files can change them to csv</p>

In [1]:
# -*- coding: utf-8 -*-
#Import packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import timeit
import codecs
from bs4 import BeautifulSoup # $ pip install beautifulsoup4

In [ ]:
#Search through the dataset directory to get all of the file directory.
mypath = 'D:\Downloads\\texts_tagged_200'
from os import walk
files = []
fileName = []
d = [mypath]
for (dirpath, dirnames, filenames) in walk(mypath):
    if(dirpath)!='./': dirpath+="/"
    files.extend([dirpath+s for s in filenames])
    fileName.extend([s for s in filenames])


In [ ]:
print(files[0]) #sample
print(len(files))

<h3>First step</h3>
<p>We will create a function that accept file directory, then read it and find any word and form them as sentence</p>

In [ ]:
def xml2text(directory):
    file = codecs.open(directory,'r','utf-8')
    fileString = file.read()
    soup = BeautifulSoup(fileString,'lxml')
    sentences = soup.find_all("se")
    all_sentences = []
    for sentence in sentences:
        words = sentence.findAll('w')
        wordsList = [word.getText() for word in words]
        if(len(wordsList)>3): all_sentences.append(wordsList)
    return all_sentences
xml2text(files[5])

<h3>Apply this to all of the files.</h3>

In [ ]:
sentences = []
start = timeit.default_timer()
for file in files:
    all_sentences = xml2text(file)
    sentences.extend(all_sentences)
stop = timeit.default_timer()
print("Execution time : "+str(round(stop - start,2))+"s.")

In [ ]:
def construct_sentence_df(files):
    sentences = []
    filePath = []
    
    for file in files[0:5]:
        all_sentences = xml2text(file)
        sentences.extend(all_sentences)
        filPath.extend([file]*len(all_sentences))
    
start = timeit.default_timer()
#construct_sentence_df(files) Alreacy created
stop = timeit.default_timer()
print("Execution time : "+str(round(stop - start,2))+"s.")

In [ ]:
len(sentences)

In [ ]:
sentences[10:20]

<h2>Next is word2vec!</h2>
<p>After we have obtain sentences, we will train it using gensim word2vec</p>

In [ ]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 500    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)




In [ ]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
#model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "models/thai_tagged_context.h5"
model.save(model_name)

In [3]:
import gensim
model = gensim.models.Word2Vec.load("models/thai_tagged_context.h5")
model.most_similar("แมว")

C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[('สุนัข', 0.5813488960266113),
 ('หมา', 0.5593259930610657),
 ('หมี', 0.5326168537139893),
 ('กระต่าย', 0.5128886103630066),
 ('หมาป่า', 0.5018278360366821),
 ('จิ้งจก', 0.4966149628162384),
 ('นกยูง', 0.48666509985923767),
 ('ม้าน้ำ', 0.4808809161186218),
 ('แพนด้า', 0.47525882720947266),
 ('ค่าง', 0.4679175317287445)]

In [4]:
model.most_similar('นักข่าว')

[('ผู้สื่อข่าว', 0.7021355628967285),
 ('คอลัมนิสต์', 0.5769671201705933),
 ('สื่อมวลชน', 0.5283889770507812),
 ('หนังสือพิมพ์', 0.5068046450614929),
 ('บรรณาธิการ', 0.5045503377914429),
 ('ไทยรัฐ', 0.4857231378555298),
 ('เดลินิวส์', 0.4820653796195984),
 ('นายแบบ', 0.4544511139392853),
 ('นักการเมือง', 0.441914826631546),
 ('โฆษก', 0.43304669857025146)]

In [6]:
model.most_similar('สวย')

[('งดงาม', 0.41707637906074524),
 ('วงศ์วาน', 0.40674978494644165),
 ('หมดจด', 0.3905469477176666),
 ('เซ็กซี่', 0.37484011054039),
 ('เฉิดฉาย', 0.3676585853099823),
 ('ดี', 0.36575961112976074),
 ('ฉูดฉาด', 0.3633784353733063),
 ('สวยๆ', 0.36257246136665344),
 ('สง่า', 0.35836389660835266),
 ('สดใส', 0.35804134607315063)]

In [12]:
model.most_similar('คุณ')

[('เธอ', 0.3916650116443634),
 ('พี่', 0.38157570362091064),
 ('ปู่', 0.3776712417602539),
 ('ป้า', 0.34502461552619934),
 ('คุณหญิง', 0.34252119064331055),
 ('พระคุณ', 0.34112977981567383),
 ('แม่', 0.33915263414382935),
 ('ท่าน', 0.33747851848602295),
 ('ลูกๆ', 0.32694211602211),
 ('ขรัว', 0.31382665038108826)]

In [13]:
model.most_similar('แต่')

[('แต่ว่า', 0.5858616232872009),
 ('เพราะ', 0.4237790107727051),
 ('แค่', 0.37721535563468933),
 ('อะไร', 0.35391560196876526),
 ('เป็นไร', 0.34483757615089417),
 ('หาก', 0.33637097477912903),
 ('ถ้า', 0.3362531363964081),
 ('จะ', 0.33621129393577576),
 ('ไม่', 0.320434033870697),
 ('แยแส', 0.3190068304538727)]

In [14]:
model.most_similar('จีบ')

[('นุ่ง', 0.4197158217430115),
 ('ไฮโซ', 0.4026893973350525),
 ('ผ้าถุง', 0.38462865352630615),
 ('ห้าว', 0.3827100694179535),
 ('ง้อ', 0.37864500284194946),
 ('ปากจัด', 0.3751307427883148),
 ('โอ๋', 0.3744509816169739),
 ('กระโปรง', 0.36587175726890564),
 ('เม้าท์', 0.3618544638156891),
 ('เกี้ยว', 0.35925015807151794)]

In [15]:
#Extract Name
model.most_similar('ธีร')

[('พัฒน', 0.6001747846603394),
 ('กีรติ', 0.5707467794418335),
 ('โสภณ', 0.5500040054321289),
 ('ธีระ', 0.5460876226425171),
 ('วัฒน์', 0.545250415802002),
 ('ณัฐ', 0.5235425233840942),
 ('รัตน์', 0.5181669592857361),
 ('นันท์', 0.5168009400367737),
 ('วัฒน', 0.5159958600997925),
 ('วิชิต', 0.5144820809364319)]

In [16]:
model.most_similar('กีรติ')

[('วัฒน', 0.5842046737670898),
 ('ธีร', 0.5707468390464783),
 ('โสภณ', 0.5637062191963196),
 ('พิศาล', 0.5493921041488647),
 ('พัฒน', 0.5462669730186462),
 ('จิตรา', 0.5301538109779358),
 ('วิมล', 0.5288647413253784),
 ('กาญจนา', 0.5240896940231323),
 ('วิชิต', 0.5240508317947388),
 ('นร', 0.5219891667366028)]

In [24]:
#Extracting name by traversing similar terms from "ธีร"
start = timeit.default_timer()

nameList = ['ธีร']
for searchingName in nameList:
    for (name,prob) in model.most_similar(searchingName):
        if(name not in nameList and prob>0.5): nameList.append(name)
stop = timeit.default_timer()
print("Execution time : "+str(round(stop - start,2))+"s.")
nameList

Execution time : 52.65s.


['ธีร',
 'พัฒน',
 'กีรติ',
 'โสภณ',
 'ธีระ',
 'วัฒน์',
 'ณัฐ',
 'รัตน์',
 'นันท์',
 'วัฒน',
 'วิชิต',
 'พัฒน์',
 'อดุล',
 'ศิริ',
 'ปิยะ',
 'พิศาล',
 'ยุคล',
 'จิตรา',
 'วิมล',
 'กาญจนา',
 'นร',
 'เมธา',
 'โกวิท',
 'สุทธ',
 'พุทธิ',
 'ธาดา',
 'โรจน์',
 'จรูญ',
 'วีระ',
 'เศรษฐ์',
 'จตุพร',
 'ศุภ',
 'ธวัช',
 'วิโรจน์',
 'วัจน์',
 'ชวลิต',
 'ณัฐวุฒิ',
 'ไพโรจน์',
 'จุฑา',
 'นิภา',
 'จินดา',
 'ตันติ',
 'ภรณ์',
 'อิศร',
 'ณรงค์',
 'ฤทธิ',
 'พงศ์',
 'ทิพ',
 'ภูวนาถ',
 'ปิย',
 'อมร',
 'วิเชียร',
 'พิมล',
 'ดิลก',
 'เศวต',
 'มุนี',
 'บริพัตร',
 'ฑิฆัม',
 'ภาณุ',
 'ฐิติ',
 'จุฬาภรณ์',
 'ศศิ',
 'จำรัส',
 'วิลาส',
 'สมันต์',
 'อุไร',
 'วิกรม',
 'หม่อมหลวง',
 'เกษมสันต์',
 'ทธา',
 'เสนี',
 'รุจิ',
 'พิทย',
 'นุสรณ์',
 'วงษ์',
 'พิริย',
 'โชติ',
 'โอภาส',
 'ดนุ',
 'อรรถ',
 'ภัทร',
 'วุฒิชัย',
 'บรรหาร',
 'อานันท์',
 'จาตุรนต์',
 'อภิรักษ์',
 'ปรีดี',
 'เนวิน',
 'อดุลย์',
 'นพดล',
 'ประโคน',
 'พฤฒิ',
 'ดนัย',
 'สุภา',
 'มานิต',
 'อัศว',
 'วานิช',
 'วิวัฒน์',
 'ทธิ์',
 'รางค์',
 'วิทย์',
 'ทัต',
 'ว

In [23]:
len(nameList)

8287

In [25]:
model.most_similar('ชั่วโมง')

[('นาที', 0.6345674395561218),
 ('สัปดาห์', 0.626414954662323),
 ('โมง', 0.5686160326004028),
 ('วินาที', 0.5139205455780029),
 ('เดือน', 0.5114910006523132),
 ('ปี', 0.5037354230880737),
 ('กิโล', 0.468478262424469),
 ('ไมล์', 0.4590434730052948),
 ('เมตร', 0.4394776225090027),
 ('วัน', 0.4292236864566803)]

In [28]:
#Try to find out meaning of unknown word
model.most_similar('ธีรภัทร')

KeyError: "word 'ธีรภัทร' not in vocabulary"

In [42]:
new_sentences = ['ธีรภัทร กำลัง จะ ไป เรียน พรุ่งนี้'.split()]
print(new_sentences)
model.build_vocab(new_sentences, update=True)
model.train(new_sentences)

[['ธีรภัทร', 'กำลัง', 'จะ', 'ไป', 'เรียน', 'พรุ่งนี้']]


22

[('นั้น', 0.7114439010620117),
 ('บาง', 0.4614031910896301),
 ('หลาย', 0.42417851090431213),
 ('แต่ละ', 0.3978354334831238),
 ('หลายๆ', 0.38571304082870483),
 ('แรก', 0.3548297584056854),
 ('ที่', 0.35286062955856323),
 ('ก่อนๆ', 0.34604236483573914),
 ('ทุก', 0.34400978684425354),
 ('นั้นๆ', 0.3373042643070221)]